# Final Project

This file reads data from ...

### Finding the columns in the database

In [20]:
import psycopg2
import pandas as pd
import requests
from bs4 import BeautifulSoup
from sqlalchemy import create_engine

In [18]:
# Database connection parameters
host = "pinot.cs.ucsb.edu"
database = "pinot"
user = "cs190nf24g11"
password = "Kp7449Jv9ZPD"
port = "5432"

## Get data from database

In [31]:
try:
    # Connect to PostgreSQL database using psycopg2
    conn = psycopg2.connect(host=host, database=database, user=user, password=password)
    print("Connection successful\n")

    # Create a cursor to interact with the database
    cursor = conn.cursor()

    # Fetch data from the 'devices' table
    query_devices = "SELECT * FROM devices;"
    cursor.execute(query_devices)
    devices_results = cursor.fetchall()

    # Fetch data from the 'wifistats' table
    query_wifistats = "SELECT * FROM wifistats;"
    cursor.execute(query_wifistats)
    wifistats_results = cursor.fetchall()

    # Convert both results to Pandas DataFrames
    devices_columns = ['label', 'ethernet_mac', 'wlan_mac', 'location', 'tags', 'comment', 'uptime', 'wireless_bytes', 'wired_bytes', 'last_seen']
    wifistats_columns = ['label', 'time', 'ping', 'signal', 'tx_bitrate', 'rx_bitrate', 'connected_time', 'packet_loss']

    devices_df = pd.DataFrame(devices_results, columns=devices_columns)
    wifistats_df = pd.DataFrame(wifistats_results, columns=wifistats_columns)

    # Print column names to verify
    print("Devices DataFrame columns:", devices_df.columns)
    print("WiFiStats DataFrame columns:", wifistats_df.columns)

    # Ensure both 'label' columns are of the same type (string) and strip any extra spaces
    devices_df['label'] = devices_df['label'].astype(str).str.strip()  # Remove extra spaces
    wifistats_df['label'] = wifistats_df['label'].astype(str).str.strip()

    # Check for missing values in 'label' columns
    print("Devices DataFrame missing 'label' values:", devices_df['label'].isnull().sum())
    print("WiFiStats DataFrame missing 'label' values:", wifistats_df['label'].isnull().sum())

    # Drop rows where 'label' is NaN (optional, depending on your needs)
    devices_df = devices_df.dropna(subset=['label'])
    wifistats_df = wifistats_df.dropna(subset=['label'])

    # Combine both tables into a single DataFrame using an outer join on 'label'
    df_combined = pd.merge(devices_df, wifistats_df, on="label", how="outer")

    # Print merged DataFrame shape and first few rows to confirm successful merge
    print(f"Combined DataFrame shape: {df_combined.shape}")
    print(df_combined.head())

    print("Data successfully merged.\n")

except Exception as e:
    print(f"Database error: {e}")
    df_combined = pd.DataFrame()  # Empty DataFrame to avoid breaking the rest of the script

finally:
    if conn:
        conn.close()  # Close the connection
        print("Connection closed\n")

    print("Script execution completed.\n")

# try:
#     # Create SQLAlchemy engine
#     engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{database}")
#     print("Database connection successful\n")

#     # Fetch data from the 'devices' table
#     devices_query = "SELECT * FROM devices;"
#     devices_df = pd.read_sql_query(devices_query, engine)

#     # Fetch data from the 'wifistats' table
#     wifistats_query = "SELECT * FROM wifistats;"
#     wifistats_df = pd.read_sql_query(wifistats_query, engine)
#     print(devices_df.columns)
#     print(wifistats_df.columns)

#     # Combine both tables into a single DataFrame (assuming they have a common 'label' column)
#     df_combined = pd.merge(devices_df, wifistats_df, on="label", how="outer")

#     print("Data successfully merged.\n")

# except Exception as e:
#     print(f"Database error: {e}")
#     df_combined = pd.DataFrame()  # Empty DataFrame to avoid breaking the rest of the script

# finally:
#     print("Script execution completed.\n")


Connection successful

Devices DataFrame columns: Index(['label', 'ethernet_mac', 'wlan_mac', 'location', 'tags', 'comment',
       'uptime', 'wireless_bytes', 'wired_bytes', 'last_seen'],
      dtype='object')
WiFiStats DataFrame columns: Index(['label', 'time', 'ping', 'signal', 'tx_bitrate', 'rx_bitrate',
       'connected_time', 'packet_loss'],
      dtype='object')
Devices DataFrame missing 'label' values: 0
WiFiStats DataFrame missing 'label' values: 0
Combined DataFrame shape: (4704663, 17)
                     label       ethernet_mac           wlan_mac location  \
0  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
1  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
2  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
3  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
4  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   

   tags comment                   

In [34]:
# remove places with no location data
df_combined = df_combined.dropna(subset=['location'])
print(df_combined.head(10))

                     label       ethernet_mac           wlan_mac location  \
0  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
1  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
2  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
3  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
4  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
5  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
6  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
7  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
8  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   
9  raspi-dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:64  dc:a6:32:d7:6e:65     None   

   tags comment                                             uptime  \
0  None    None   18:11:40 up 206 days, 17:13,  0 users,  load ...   
1  None    N

## Find Locations with Poorest Connecitity

In [37]:
try:
    # Clean data: Convert necessary columns to numeric
    df_combined['packet_loss'] = pd.to_numeric(df_combined['packet_loss'], errors='coerce')
    df_combined['signal'] = pd.to_numeric(df_combined['signal'], errors='coerce')
    df_combined['tx_bitrate'] = pd.to_numeric(df_combined['tx_bitrate'], errors='coerce')
    df_combined['rx_bitrate'] = pd.to_numeric(df_combined['rx_bitrate'], errors='coerce')
    
    # Calculate a connectivity score (lower score = worse connectivity)
    # Higher packet_loss, lower signal, and lower bitrate are worse
    df_combined['connectivity_score'] = (
        df_combined['packet_loss'] * 2 +  # Weight packet loss more
        (1 / (df_combined['signal'] + 1e-5)) +  # Lower signal is worse, adding a small value to avoid division by zero
        (1 / (df_combined['tx_bitrate'] + 1e-5)) +  # Lower tx_bitrate is worse
        (1 / (df_combined['rx_bitrate'] + 1e-5))    # Lower rx_bitrate is worse
    )
    
    # Remove duplicates based on 'label' and 'location', keeping the row with the highest connectivity score
    df_combined_unique = df_combined.drop_duplicates(subset=['label', 'location'], keep='first')
    
    # Sort by the connectivity score (descending: higher score = worse connectivity)
    df_sorted = df_combined_unique.sort_values(by='connectivity_score', ascending=False)
    
    # Get the top 10 places with the poorest connectivity
    top_10_poor_places = df_sorted[['label', 'location', 'connectivity_score']].head(10)
    
    print("Top 10 places with the poorest connectivity:")
    print(top_10_poor_places)
    
    print("Data successfully processed.\n")

except Exception as e:
    print(f"Database error: {e}")
    df_combined = pd.DataFrame()  # Empty DataFrame to avoid breaking the rest of the script

finally:
    if conn:
        conn.close()  # Close the connection
        print("Connection closed\n")

    print("Script execution completed.\n")


Top 10 places with the poorest connectivity:
                          label              location  connectivity_score
120426  raspi-e4:5f:01:75:54:ec          Rincon, 2508                 NaN
198882  raspi-e4:5f:01:ad:c9:2a       La Cumbre, 2207                 NaN
261302  raspi-e4:5f:01:a7:b1:c1       Tepusquet, 2408                 NaN
339706  raspi-e4:5f:01:8d:31:8b   SNL Lab Server Room                 NaN
339708  raspi-e4:5f:01:72:a2:eb          Encino, 5119                 NaN
390021  raspi-e4:5f:01:a0:35:20   SNL Lab Server Room                 NaN
390022  raspi-e4:5f:01:8c:38:b1      San Miguel, 2225                 NaN
405189  raspi-e4:5f:01:ac:e4:08   SNL Lab Server Room                 NaN
405190  raspi-e4:5f:01:ad:c9:96   SNL Lab Server Room                 NaN
405191  raspi-e4:5f:01:a7:b1:a3  Santa Catalina, 0234                 NaN
Data successfully processed.

Connection closed

Script execution completed.

